In [1]:
import pandas as pd
cps = pd.read_stata('https://github.com/nickeubank/MIDS_Data/blob/master/Current_Population_Survey/morg18.dta?raw=true')

In [4]:
cps.ethnic.value_counts(dropna=False, normalize=True).sort_index()

1.0    0.079628
2.0    0.011259
3.0    0.005054
4.0    0.004277
5.0    0.004823
6.0    0.007154
7.0    0.008345
8.0    0.009192
NaN    0.870268
Name: ethnic, dtype: float64